# SARSA (Estado-Acción-Recompensa-Estado-Acción)   (Aprendizaje por Reforzamiento)

SARSA proviene de las siglas en inglés de la expresión "State-Action-Reward-State-Action", lo cual describe el proceso que sigue el algoritmo al actualizar los valores Q.

La principal diferencia es que SARSA toma en cuenta tanto el estado actual como la acción actual para predecir la próxima acción y su recompensa, lo que lo hace un algoritmo on-policy.

Alpha es la tasa de aprendizaje que ajusta la rapidez con la que el agente actualiza sus estimaciones de valor Q en respuesta a nueva información.

Epsilon regula la probabilidad de que el agente elija una acción al azar, fomentando la exploración de nuevas acciones frente a la explotación del conocimiento adquirido.

La función estado_a_indice(estado) toma un estado en dos dimensiones y lo convierte en un índice único para poder ubicarlo en la tabla Q.


In [1]:
import numpy as np 
import random 
import matplotlib.pyplot as plt

In [2]:
dimensiones = (4,4)
estado_inicial = (0,0)
estado_objetivo = (3,3)
acciones = [(0,-1),(0,1),(-1,0),(1,0)]
acciones_simbolos = ['↑','↓','←','→']

In [3]:
num_estados = dimensiones[0] * dimensiones[1]
num_estados #Número de estados en el entorno

16

In [4]:
num_acciones = len(acciones)
print(f'Número de acciones posibles: {num_acciones}') #Número de acciones posibles(arriba, abajo, izq, der.)

Número de acciones posibles: 4


In [5]:
Q = np.zeros((num_estados, num_acciones))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [6]:
alpha = 0.1 #Tasa de Aprendizaje
gamma = 0.99 #Factor de Descuento
epsilon = 0.2 #Tasa de exploración
episodios = 1000 #Mientras mas alto el valor mas aprende


In [7]:
def estado_a_indice(estado):
    return estado[0] * dimensiones[1] + estado[1]

In [8]:
estado_a_indice((3,0))

12

In [9]:
def elegir_accion(estado):
    if random.uniform(0,1) < epsilon:
        return random.randint(0, num_acciones -1)
    else:
        return np.argmax(Q[estado_a_indice(estado)])

np.add(estado, accion) suma la posición actual del agente con el cambio indicado por la acción para obtener la nueva posición.

El operador % aplica límites "atravesables", permitiendo que la posición se envuelva al otro lado de la cuadrícula si se excede.(np.add(estado, accion) % np.array(dimensiones))



En cada paso del bucle, se actualiza el valor de la matriz Q con la fórmula de actualización Q-learning basada en la recompensa recibida y las acciones futuras esperadas. (while not terminado)

In [12]:
def aplicar_accion(estado, accion_idx):
    accion = acciones[accion_idx]
    nuevo_estado = tuple(np.add(estado, accion) % np.array(dimensiones))
    
    if nuevo_estado == estado_objetivo:
        recompensa = 1
    else:
        recompensa = -1
        
    return nuevo_estado, recompensa, nuevo_estado == estado_objetivo
        

In [13]:
for episodio in range(episodios):
    estado = estado_inicial
    accion_idx = elegir_accion(estado)
    terminado = False
    
    while not terminado: 
        nuevo_estado, recompensa, terminado = aplicar_accion(estado, accion_idx)
        nueva_accion_idx = elegir_accion(nuevo_estado)
        
        indice = estado_a_indice(estado)
        Q[indice, accion_idx] += alpha * (recompensa + gamma * Q[estado_a_indice(nuevo_estado), nueva_accion_idx] - Q[indice, accion_idx])
        
        estado, accion_idx = nuevo_estado, nueva_accion_idx

In [14]:
politica_simbolos = np.empty(dimensiones, dtype='<U2')
politica_simbolos

array([['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', '']], dtype='<U2')

politica_simbolos es utilizada para visualizar las mejores acciones en cada estado con símbolos como flechas.

np.argmax(Q[estado_a_indice(estado)]) encuentra el índice de la acción con el mayor valor Q para un estado específico, indicando la mejor acción a tomar.

In [16]:
for i in range(dimensiones[0]):
    for j in range(dimensiones[1]):
        estado = (i, j)
        mejor_accion = np.argmax(Q[estado_a_indice(estado)])
        politica_simbolos[i, j] = acciones_simbolos[mejor_accion]
politica_simbolos        

array([['←', '→', '←', '←'],
       ['↓', '←', '→', '→'],
       ['→', '→', '→', '→'],
       ['↑', '↓', '↓', '↑']], dtype='<U2')